In [1]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
from math import *
from sklearn.cluster import KMeans
from sklearn.metrics import *
from pathlib import Path

ModuleNotFoundError: No module named 'cv2'

In [3]:
def bresenham(start, end):
    """Bresenham's Algorithm
    Produces a list of tuples from start and end
 
    >>> points1 = get_line((0, 0), (3, 4))
    >>> points2 = get_line((3, 4), (0, 0))
    >>> assert(set(points1) == set(points2))
    >>> print points1
    [(0, 0), (1, 1), (1, 2), (2, 3), (3, 4)]
    >>> print points2
    [(3, 4), (2, 3), (1, 2), (1, 1), (0, 0)]
    """
    # Setup initial conditions
    x1, y1 = start
    x2, y2 = end
    dx = x2 - x1
    dy = y2 - y1
 
    # Determine how steep the line is
    is_steep = abs(dy) > abs(dx)
 
    # Rotate line
    if is_steep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
 
    # Swap start and end points if necessary and store swap state
    swapped = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        swapped = True
 
    # Recalculate differentials
    dx = x2 - x1
    dy = y2 - y1
 
    # Calculate error
    error = int(dx / 2.0)
    ystep = 1 if y1 < y2 else -1
 
    # Iterate over bounding box generating points between start and end
    y = y1
    points = []
    for x in range(x1, x2 + 1):
        coord = (y, x) if is_steep else (x, y)
        points.append(coord)
        error -= abs(dy)
        if error < 0:
            y += ystep
            error += dx
 
    # Reverse the list if the coordinates were swapped
    if swapped:
        points.reverse()
    return points

In [4]:
# Generique Fourrier Descriptor
# Argument : The opencv image, 
# Return : GFD Signature
def GFD(image, m, n):
    N = image.shape[1]
    maxR = sqrt((((N)//2)**2) + (((N)//2)**2))

    x = np.linspace(-(N-1)//2, (N-1)//2, N)
    y = x
    X, Y = np.meshgrid(x, y)

    radius = np.sqrt(np.power(X, 2) + np.power(Y, 2)) / maxR

    theta = np.arctan2(Y, X)
    theta[theta < 0] = theta[theta < 0] + (2 * np.pi)

    FR = np.zeros((m,n))
    FI = np.zeros((m,n))
    FD = np.zeros((m*n,1))

    i = 0
    for rad in range(m):
        for ang in range(n):
            tempR = image.dot(np.cos(2 * np.pi * rad * radius + ang * theta))
            tempI = image.dot(np.sin(2 * np.pi * rad * radius + ang * theta))
            FR[rad, ang] = np.sum(tempR)
            FI[rad, ang] = np.sum(tempI)
            
            if rad == 0 and ang == 0:
                FD[i] = sqrt((2* (FR[0,0] * FR[0,0]))) / (np.pi* maxR * maxR)
            else:
                FD[i] = sqrt((FR[rad, ang] * FR[rad, ang]) + (FI[rad, ang] * FI[rad, ang])) / (sqrt((2* (FR[0,0] * FR[0,0]))))
            i = i + 1
    return FD

In [5]:
#penser a refactorer ce bloc si on a assez de tps
def distance(a,b):
    x1 = a[0]
    x2 = b[0]
    y1 = a[1]
    y2 = b[1]
    return sqrt((x1-x2)**2+(y1-y2)**2)

def sumDistance(a,points):
    sum = 0
    for point in points:
        sum += distance(a,point)
    return sum

def heron(a,b,c):
    s = (a + b +c)/2 
    try:
        res = sqrt(s*(s-a)*(s-b)*(s-c))
    except ValueError:
        return 0
    else:
        return res

def equaDist(a,b,c):
    return (isclose(distance(a,b),distance(b,c),rel_tol = 0.07))

def is_equa_dist(a,b,c):
        #isocele en b
    if (equaDist(a,b,c) and not (equaDist(b,c,a))):
        return True
    #isocele en c
    elif (equaDist(b,c,a) and not (equaDist(c,a,b))):
        return True
    #isocele en a
    elif (equaDist(c,a,b) and not (equaDist(a,b,c))):
        return True
    else :
        return False

def isIsoceles(points,nbrPoints = 3):
    if(nbrPoints == 3):
        a = points[0][0]
        b = points[1][0]
        c = points[2][0]
        return is_equa_dist(a,b,c)
    
    elif(nbrPoints == 4):
        l1 = points[1:4,0]   
        sum_d1 = sumDistance(points[0][0],l1)

        l2 = points[0:1,0]+points[2:4,0]
        sum_d2 = sumDistance(points[1][0],l2)

        l3 = points[0:2,0]+points[3:4,0]
        sum_d3 = sumDistance(points[2][0],l3)

        l4 = points[0:3,0]
        sum_d4 = sumDistance(points[3][0],l4)
        
        dl = [sum_d1,sum_d2,sum_d3,sum_d4]

        if(dl.index(min(dl)) == 0):
            a = points[3][0]
            b = points[1][0]
            c = points[2][0]

        elif(dl.index(min(dl)) == 1):
            a = points[0][0]
            b = points[2][0]
            c = points[3][0]
            
        elif(dl.index(min(dl)) == 2):
            a = points[0][0]
            b = points[1][0]
            c = points[3][0]
    
        elif(dl.index(min(dl)) == 3):
            a = points[0][0]
            b = points[1][0]
            c = points[2][0]
        else :
            print("Error")
            return False
        return is_equa_dist(a,b,c)
    else :
        print("Error")
        return False    

def heronPoints(points,nbrPoints = 3):
    #on suppose qu'on aura 3 points en entrée
    if(nbrPoints == 3):
        d1 = distance(points[0][0],points[1][0])
        d2 = distance(points[1][0],points[2][0])
        d3 = distance(points[2][0],points[0][0])

        return heron(d1,d2,d3)   
    
    #on suppose qu'on aura 4 points en entrée
    elif(nbrPoints == 4):
        l1 = points[1:4,0]   
        sum_d1 = sumDistance(points[0][0],l1)

        l2 = points[0:1,0]+points[2:4,0]
        sum_d2 = sumDistance(points[1][0],l2)

        l3 = points[0:2,0]+points[3:4,0]
        sum_d3 = sumDistance(points[2][0],l3)

        l4 = points[0:3,0]
        sum_d4 = sumDistance(points[3][0],l4)

        dl = [sum_d1,sum_d2,sum_d3,sum_d4]

        if(dl.index(min(dl)) == 0):
            d1 = distance(points[1][0],points[2][0])
            d2 = distance(points[2][0],points[3][0])
            d3 = distance(points[3][0],points[1][0])
            
        elif(dl.index(min(dl)) == 1):
            d1 = distance(points[0][0],points[2][0])
            d2 = distance(points[2][0],points[3][0])
            d3 = distance(points[3][0],points[0][0])

        elif(dl.index(min(dl)) == 2):
            d1 = distance(points[0][0],points[1][0])
            d2 = distance(points[1][0],points[3][0])
            d3 = distance(points[3][0],points[0][0])

        elif(dl.index(min(dl)) == 3):
            d1 = distance(points[0][0],points[1][0])
            d2 = distance(points[1][0],points[2][0])
            d3 = distance(points[2][0],points[0][0])
    
        else :
            print("Error")
            return -1
        return heron(d1,d2,d3)
    else :
        print("Error")
        return -1

In [2]:
# Argument : the folder path 
# Return : an array of all images readed by OpenCV
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), 0) # Gray Color converting
        if img is not None:
            images.append(img)
    return np.array(images)

# Elbow Method
def elbow_method(X):
    inertia = []
    K_range = range(1, 10)
    for k in K_range:
        model = KMeans(n_clusters=k).fit(X)
        inertia.append(model.inertia_)
    plt.plot(K_range, inertia)
    plt.xlabel('nombre de clusters')
    plt.ylabel('cout du model (inertia)')

def euclidean_distance(vec1, vec2):
    return np.sum(np.sqrt((vec1 - vec2)**2))

# Return bool if the contour is a triangle or not 
def is_triangle(contour):
    peri = cv2.arcLength(contour,True)
    vertices = cv2.approxPolyDP(contour,0.05*peri,True)
    nbr_points = len(vertices)
    if (nbr_points == 4) or(nbr_points== 3):
        area = cv2.contourArea(contour)
        heronVal = heronPoints(vertices,nbr_points)
        return (0.70*heronVal<area < 1.3*heronVal) and isIsoceles(vertices,nbr_points)
    return False

## Binarisation des images

In [6]:
# Voir : https://www.programcreek.com/python/example/70440/cv2.findContours

images = load_images_from_folder("./Ressources/test")

for image in images:
    # Denoising the image
    image = cv2.GaussianBlur(image, (5, 5), 3)
    
    # Binarisation et utilisation de OTSU pour déterminer le seuil automatiquement
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU) # Ne detecte que les flèches noires, il faut modifier le param 2 et 3 pour inverser cela et ajouter l'inverse de l'image
        
    # Contours Detection
    edged = cv2.Canny(binary,10,200)

    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # Get only extreme points of the contours
    
    # Sort the contours by area and define le threshold area min
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    threshold_area_min = cv2.contourArea(contours[0])*0.05

    num = 0
    descriptors = []
    rois = []

    for i, c in enumerate(contours):
        x,y,w,h = cv2.boundingRect(c)

        if is_triangle(c) and cv2.contourArea(c) > threshold_area_min:
            
            ROI = binary[y:y+h, x:x+w]

            # Add white border
            ROI = np.pad(ROI, pad_width=4, mode='constant', constant_values=255)

            edged = cv2.Canny(binary,10,200)
            #edged = cv2.convertScaleAbs(cv2.Laplacian(ROI,cv2.CV_64F))

            contours2, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            biggest = max(contours2, key=cv2.contourArea)

            im = np.copy(ROI)

            #mask = np.ones(ROI.shape[:2], np.uint8)
            cv2.drawContours(im,[biggest],-1,(0, 0, 255),3)
            if i == 0:
                print(len(contours2))
                plt.imshow(im, cmap='gray')
        
            # make here the shape detection and filter ROIs

            descriptors.append(GFD(ROI, 4, 9))
            rois.append(ROI)

            #cv2.drawContours(ROI, [approx], -1, (0, 255, 0), 3)            
            cv2.imwrite('./resultats/ROI_{}.png'.format(num), ROI)
            num += 1

<ipython-input-2-8341fa3f19c4>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(images)


In [7]:
descriptors = np.array(descriptors)
rois = np.array(rois)

x, y, z = descriptors.shape
X = descriptors.reshape((x,y*z))

#elbow_method(X)

model = KMeans(n_clusters=3, init='k-means++')
labels = model.fit_predict(X)
plt.scatter(X[:,0], X[:, 1], c=labels)
plt.scatter(model.cluster_centers_[:, 0], model.cluster_centers_[:, 1])

#print(model.labels_)

for i in range (0, 2):
    for num, image in enumerate(rois[model.labels_ == i]):
        Path(f'./resultats/class_{i+1}').mkdir(exist_ok=True)
        cv2.imwrite(f'./resultats/class_{i+1}/{num}.png', image)

ValueError: not enough values to unpack (expected 3, got 1)

In [83]:
if cv2.waitKey(0) & 0xFF == ord('q'):
    cv2.destroyAllWindows()